# Example for usage ARc Policy via a tool for a Strands Agent

Install requirements

In [ ]:
%pip install -r requirements.txt --upgrade

In [ ]:
# If needed, this will allow for the restart of the notebook

from IPython.display import display_html
def restart_kernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)
restart_kernel()

## Setup for the agent

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent
from pydantic import BaseModel
import boto3
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
StrandsTelemetry().setup_console_exporter()

# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used

In [ ]:
# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ID = "<REPLACE ME>"
GUARDRAIL_ID = "<REPLACE ME>"
GUARDRAIL_VERSION = "<REPLACE ME>"
# NOTE: the default model for Strands is us.anthropic.claude-sonnet-4-20250514-v1:0
MODEL_ID = "us.amazon.nova-lite-v1:0"

In [ ]:

# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
class NotifyOnlyGuardrailsHook(HookProvider):
    
    def __init__(self, guardrail_id: str, guardrail_version: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''
        self.response_valid = True

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(MessageAddedEvent, self.process_message)


    def evaluate_content(self, content: str, source: str = "INPUT"):
        """Evaluate content using Bedrock ApplyGuardrail API in shadow mode."""

        # User input, no need to call the guardrail in this case yet
        if source == "INPUT":
            self.input = content
            return
        
        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]
        
        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        # Determine if the output is safe
        assessments = response.get("assessments", [])
        if assessments and len(assessments):
            arcpolicy = assessments[0].get("automatedReasoningPolicy", [])

            if arcpolicy and len(arcpolicy):
                findings = arcpolicy.get("findings", [])

                if findings and len(findings):
                    self.response_valid = False
        
    def process_message(self, event: MessageAddedEvent) -> None:
        """Check message and determine if it's user or assistant created."""

        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        source = "INPUT"
        if event.message.get("role") == "assistant":
            source = "OUTPUT"
        
        # Call the eval function to determine if something should be done
        self.evaluate_content(content, source)

        # In the case of output and the arc findings are in place, signal that to the caller
        if source == "OUTPUT":
            # Add information to the output
            new_output = content + f"\n\nresponse_valid: {self.response_valid}"
            event.message["content"][0]["text"] = new_output

# Create structured output
class ArcValidationCheck(BaseModel):
    response_valid: bool
    content: str

# Provide the config for botocore
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    model_id=MODEL_ID,
    # NOTE: An alternative option is to supply the guardrail here.  If going that route, the ARc findings aren't present.
    # To ensure that the findings are present and can be used to re-write the output, rely on a hook
)

# Create agent with the guardrail-protected model
auto_policy_agent = Agent(
    system_prompt="You are a helpful assistant that will answer questions related to auto insurance policy.",
    model=bedrock_model,
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION)],
)



In [ ]:
# Use the protected agent for conversations
auto_policy_agent("If I have a trailer and I get into an accident is that considered a separate vehicle?")

# Get the structured output.  Without providing a prompt, the agent will act on conversation history to get the response
response = auto_policy_agent.structured_output(ArcValidationCheck)

# Extract the findings from the response
print(f"response.response_valid = {response.response_valid}")
print(f"repsonse.content = {response.content}")